In [1]:
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

In [3]:
check_transactions_df= pd.read_excel('sms_data.xlsx')

In [4]:
check_transactions_df['samplesms'] = check_transactions_df['samplesms'].apply(lambda x: str(x))
check_transactions_df['regex'] = check_transactions_df['regex'].apply(lambda x: str(x))

In [5]:
check_transactions_df.shape

(52, 198)

In [6]:
check_transactions_df = check_transactions_df[~check_transactions_df.regex.isin([np.nan])]

In [ ]:
def sure_regex_checks(transactions_df):

    transactions_df['Test01'] = transactions_df.apply(lambda x: 'fail' if re.search(' o/w', x['regex'].lower())
                                                                        and (x['smstype'] != 'credit-transaction' or
                                                                        x['smssubtype'] != 'income' or x['transactiontype'] != 'cheque'
                                                                        or x['transactionchannel'] != 'other') else 'pass', axis = 1)
    print('Test01===completed===========================================================================')

    transactions_df['Test02'] = transactions_df.apply(lambda x: 'fail' if re.search('i/w', x['regex'].lower())
                                                                        and (x['smstype'] != 'debit-transaction' or
                                                                        x['smssubtype'] != 'expense' or x['transactiontype'] != 'cheque'
                                                                        or x['transactionchannel'] != 'other') else 'pass', axis = 1)
    print('Test02===completed===========================================================================')

# If ‘IMPS/NEFT/RTGS’ and ‘debited’ in the SMS body then: SMS Type - debit, Account Type- Bank Account, SMS Sub type - Transfer out,  Transaction Channel - Net Banking
    transactions_df['Test03'] = transactions_df.apply(lambda x: 'fail' if re.search('imps|neft|rtgs|p2p|p2a', x['regex'].lower())
                                                                        and re.search('debit|debited', x['regex'].lower())
                                                                         and (x['smstype'] != 'debit-transaction'
                                                                        or x['accounttype'] != 'bank' or x['smssubtype'] != 'transfer-out'
                                                                        or x['transactionchannel'] != 'net-banking')  else 'pass', axis = 1)
    print('Test03===completed===========================================================================')

# If ‘IMPS/NEFT/RTGS’ and ‘credited’ in the SMS body then: SMS Type - credit, Account Type- Bank Account, SMS Sub type - income, Transaction Type - Account Transfer,  Transaction Channel - Net Banking
    transactions_df['Test04'] = transactions_df.apply(lambda x: 'fail' if re.search('imps|neft|rtgs', x['regex'].lower())
                                                                        and re.search('credit|credited', x['regex'].lower())
                                                                        and not re.search('debit|debited', x['regex'].lower())
                                                                       and not re.search('credited to', x['regex'].lower())
                                                               and not re.search('credited to', x['regex'].lower())
                                                                        and (x['smstype'] != 'credit-transaction'
                                                                        or x['accounttype'] != 'bank' or x['transactiontype'] != 'account-transfer'
                                                                        or x['transactionchannel'] != 'net-banking')  else 'pass', axis = 1)
    print('Test04===completed===========================================================================')
      # If smstype is bill and smssubtype is bill-payment-reminder and totalamountdue is -1 or minamountdue then fail otherwise pass
    transactions_df['Test05'] = transactions_df.apply(lambda x: 'fail' if x['smstype'] == 'bill' and x['smssubtype'] == 'bill-payment-reminder' and (x['totalamountdue'] == -1 or x['minamountdue'] == -1 )
                                                                          and x['amount'] != -1 and x['duedate'] == -1 else 'pass', axis = 1)
    print('Test05===completed===========================================================================')

   # If smstype is bill and smssubtype is bill-payment-rejection and transaction amount is -1 then fail otherwise pass
    transactions_df['Test06'] = transactions_df.apply(lambda x: 'fail' if x['smstype'] == 'bill' and x['smssubtype'] == 'bill-payment-rejection' and x['smssubtype'] != 'bill-past-due' else 'pass', axis = 1)
    print('Test06===completed===========================================================================')

   # If smstype is bill and smssubtype is bill-payment-confiramation and transaction amount is -1 then fail otherwise pass
    transactions_df['Test07'] = transactions_df.apply(lambda x: 'fail' if x['smstype'] == 'bill' and x['smssubtype'] == 'bill-payment-confirmation' and x['amount'] == -1 else 'pass', axis = 1)
    print('Test07===completed===========================================================================')

   # If smstype is credit-transaction or debit-transction and transaction amount is -1 and pan is -1 then fail otherwise pass
    transactions_df['Test08'] = transactions_df.apply(lambda x: 'fail' if (x['smstype'] == 'debit-transaction' or x['smstype'] == 'credit-transaction') and x['amount'] == -1
                                                                          and x['pan'] == -1 else 'pass', axis = 1)
    print('Test08===completed===========================================================================')
    
    transactions_df['Test09'] = transactions_df.apply(lambda x: 'fail' if (re.search('imps|neft|rtgs|fund\s*transfer|\s*FT\s*', x['samplesms'].lower())
                                                                       and x['transactionchannel']!='net-banking')
                                                                           else 'pass', axis = 1)
    print('Test09===completed===========================================================================')
    
    transactions_df['Test10'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['smssubtype']=='cc-bill-payment') and (x['accounttype']!='credit-card') else
                           'pass', axis=1)
    print('Test10===completed===========================================================================')

    transactions_df['Test11'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['accounttype']=='bank') 
                                                              and (x['smstype'] in ('cc-bill-payment','emi-installment'))else 'pass', axis=1)

    print('Test11===completed===========================================================================')

    transactions_df['Test12'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['transactiontype']=='account-transfer') 
                                                              and (x['receiveraccountid']!=-1)else 'pass', axis=1)

    print('Test12===completed===========================================================================')

    transactions_df['Test13'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction') 
                                                              and (x['minamountdue']!=-1 or x['totalamountdue']!=-1) else 'pass', axis=1)

    print('Test13===completed===========================================================================')

    transactions_df['Test14'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='credit-transaction' and x['accounttype']=='fd') 
                                                              and (x['smssubtype'] in ('refund-reversal', 'cash-back')) else 'pass', axis=1)

    print('Test14===completed===========================================================================')

    transactions_df['Test15'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['accounttype']=='credit-card') 
                                                              and (x['balance']!=-1) else 'pass', axis=1)

    print('Test15===completed===========================================================================')

    transactions_df['Test16'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['accounttype'] in ('bank','debit-card')) 
                                                              and (x['availablelimit']!=-1 ) else 'pass', axis=1)

    print('Test16===completed===========================================================================')

    transactions_df['Test17'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and x['smssubtype']=='bill-default') 
                                                              and (x['transactiontype']=='auto-debit') else 'pass', axis=1)

    print('Test17===completed===========================================================================')

    transactions_df['Test18'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['smssubtype']=='cash-withdrawal') 
                                                              and (x['transactionchannel']!='other') else 'pass', axis=1)

    print('Test18===completed===========================================================================')

    transactions_df['Test19'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['smssubtype']=='cc-bill-payment') 
                                                              and (x['accounttype']=='credit-card') else 'pass', axis=1)

    print('Test19===completed===========================================================================')


    transactions_df['Test20'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['transactiontype']=='account-transfer') 
                                                              and (x['senderaccountid']!=-1) else 'pass', axis=1)


    print('Test20===completed===========================================================================')

    transactions_df['Test21'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bank-notification' and x['smssubtype']=='beneficary-add') 
                                                              and (x['sendername']!=-1) else 'pass', axis=1)

    print('Test21===completed===========================================================================')


    transactions_df['Test22'] = transactions_df.apply(lambda x: 'fail' if (re.search('fastag' , x['regex'].lower()) and x['smstype']=='debit-transaction' and x['smssubtype']!='fastag') else 'pass', axis=1)

    print('Test22===completed===========================================================================')
    
    transactions_df['Test23'] = transactions_df.apply(lambda x: 'fail' if (re.search('bill' , x['regex'].lower())) and x['smstype']=='debit-transaction' and (x['transactiontype'] in ('regular','fine','bank-fee','cheque','cash-withdrawal')) else 'pass', axis=1)

    print('Test23===completed===========================================================================')

    transactions_df['Test24'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bank-notification' and x['smssubtype']=='loan-disbursed'
                                                              and x['amount']!=-1) else 'pass', axis=1)

    print('Test24===completed===========================================================================')

    transactions_df['Test25'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and (re.search('overdue' , x['regex'].lower()))
                                                              and x['smssubtype']=='bill-payment-confirmation') else 'pass', axis=1)

    print('Test25===completed===========================================================================')

    transactions_df['Test26'] = transactions_df.apply(lambda x: 'fail' if ((re.search('bounced' , x['regex'].lower()))
                                                              and x['transactionchannel']!='auto-debit') else 'pass', axis=1)

    print('Test26===completed===========================================================================')

    transactions_df['Test27'] = transactions_df.apply(lambda x: 'fail' if ((x['transactiontype']=='cash-withdrawal' or x['transactiontype']=='cash')
                                                              and x['transactionchannel']!='other') else 'pass', axis=1)

    print('Test27===completed===========================================================================')

    transactions_df['Test28'] = transactions_df.apply(lambda x: 'fail' if (x['accounttype']=='upi'
                                                              and x['transactionchannel']!='upi') else 'pass', axis=1)

    print('Test28===completed===========================================================================')

    transactions_df['Test29'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='beneficiary-add'
                                                              and x['accountype']!='bank') else 'pass', axis=1)

    print('Test29===completed===========================================================================')

    transactions_df['Test30'] = transactions_df.apply(lambda x: 'fail' if (re.search(' ach\s*| nach\s*| standing\s*instruction| ecs\s*| SI\s*', x['regex'].lower())
                                                              and x['transactionchannel']!='auto-debit') else 'pass', axis=1)

    print('Test30===completed===========================================================================')

    transactions_df['Test31'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='loan-disbursed'
                                                              and x['accounttype'] not in ('pay-later-account','loan','loan-against-dc','loan-against-cc','credit-line','credit-card','bullet-loan')) else 'pass', axis=1)

    print('Test31===completed===========================================================================')

    transactions_df['Test32'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='transaction-emi-convert'
                                                              and x['accounttype'] not in ('pay-later-account','loan','loan-against-dc','loan-against-cc','credit-line','credit-card','bullet-loan')) else 'pass', axis=1)

    print('Test32===completed===========================================================================')
    
    transactions_df['Test33'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='debit-transaction' and x['smssubtype']=='transfer-out'
                                                              and x['senderaccountid']!=-1) else 'pass', axis=1)

    print('Test33===completed===========================================================================')

    transactions_df['Test34'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and x['smssubtype'] in ('bill-reminder','bill-past-due')
                                                              and x['date']!=-1) else 'pass', axis=1)

    print('Test34===completed===========================================================================')

    transactions_df['Test35'] = transactions_df.apply(lambda x: 'fail' if (x['smstype']=='bill' and x['smssubtype']=='bill-payment-confirmation'
                                                              and x['duedate']!=-1) else 'pass', axis=1)

    print('Test35===completed===========================================================================')
    
    transactions_df['Test36'] = transactions_df.apply(lambda x: 'fail' if (x['transactiontype']=='salary' 
                                                              and x['accounttype']!='bank') else 'pass', axis=1)

    print('Test36===completed===========================================================================')
    
    transactions_df['Test37'] = transactions_df.apply(lambda x: 'fail' if (x['transactiontype']=='cheque' 
                                                              and x['accounttype']!='bank' and x['transactionchannel']!='other') else 'pass', axis=1)

    print('Test37===completed===========================================================================')
    
    transactions_df['Test38'] = transactions_df.apply(lambda x: 'fail' if (x['sms_tags']=='gold-loan' 
                                                              and re.search('gold\s*loan', x['regex'].lower())) else 'pass', axis=1)
    print('Test38===completed===========================================================================')
    
    transactions_df['Test39'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='balance-notification' and x['accounttype']=='bank' 
                                                              and x['balance']==-1) else 'pass', axis=1)
    print('Test39===completed===========================================================================')

    transactions_df['Test40'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype'] in ('debit-scheduled','payment-transfer-request') 
                                                              and x['totalamountdue']!=-1 and x['minamountdue']!=-1) else 'pass', axis=1)
    print('Test41===completed===========================================================================')

    transactions_df['Test42'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype'] in ('credit-card-limit-increased','credit-card-limit-exceeded','credit-card-limit-decreased') 
                                                              and x['accounttype'] not in('credit-card','credit-line','pay-later-account')) else 'pass', axis=1)
    print('Test42===completed===========================================================================')
    
    transactions_df['Test43'] = transactions_df.apply(lambda x: 'fail' if (x['smssubtype']=='balance-notification' and x['accounttype']=='credit-card' 
                                                              and x['availablelimit']==-1) else 'pass', axis=1)
    print('Test43===completed===========================================================================')


    return transactions_df

In [ ]:
check_transactions_df = sure_regex_checks(check_transactions_df)

In [ ]:
check_transactions_df.to_excel('to_check.xlsx',index=0)

# Sender service check

In [ ]:
df = pd.read_excel('sms_data.xlsx', sheet_name='result')

In [ ]:
df_multiple_service_types = df[df.groupby('senderaddress')['servicename'].transform('nunique') > 1]
df_multiple_sender_addresses = df[df.groupby('servicename')['senderaddress'].transform('nunique') > 1]

In [ ]:
df_violations = pd.concat([df_multiple_service_types, df_multiple_sender_addresses])

In [ ]:
df_violations.to_excel('sender_service_fail.xlsx',index=0)

# Regex Checks

In [ ]:
rule_dict = {}
for col in [x for x in check_transactions_df.columns if 'Test' in x]:
    rule_dict[col]=check_transactions_df[check_transactions_df[col]=='fail'].shape[0]

In [ ]:
rule_dict = {}
hashes = []
for col in [x for x in check_transactions_df.columns if 'Test' in x]:
    rule_dict[col]=check_transactions_df[check_transactions_df[col]=='fail'].shape[0]
    for x in check_transactions_df[check_transactions_df[col]=='fail']['hash']:
        hashes.append(x)

In [ ]:
hashes

In [ ]:
sorted(rule_dict.items() , key=lambda x:x[1], reverse = True)